In [ ]:
from os.path import join  #os.path.join
from bs4 import BeautifulSoup as bs
from google.colab import drive
import csv, sys, requests, json, re

#取得下載點的資料
def getCSV_From_URL(url):
  # header的功能?? 
  header = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}
  try:
    result = requests.get(url, headers=header, allow_redirects=True)
  except:
    result = "Exception"
    print("{} has exception!!".format(url))
  return result

#開啟檔案及轉成List
def openFile(File): # File是檔名
  with open(File) as myCSVFile:
    csvReader = csv.reader(myCSVFile, delimiter=',', quotechar = '"')
    result = list(csvReader)
  return result

#顯示出幾行資料 
def print_header_and_few_line(data, limit):
	counter = 0
	for i in data:
		if counter > limit:
			break
		print(i)
		counter += 1

#取出欄位值
def getColumnInedex():
	colIndexList = []
	indexValue = int(input("請依序輸入你想要的欄位索引值, 輸入 -1 結束："))
	
	while indexValue >= 0 :
		colIndexList.append(indexValue)
		indexValue = int(input("請依序輸入你想要的欄位索引值, 輸入 -1 結束："))
	return colIndexList

#產生一行的SQL語法
def genSQLValues(targetRow, selectedColumnList):
	result = "("
	for i in selectedColumnList:
		result += "\"{}\",".format(targetRow[i])
  
	result = result[:-1] + "),"
	return result

#產生全部的SOL語法
def gen_SQL_insert_cmd(tablename, data, colIndex):
  result = []
  result.append("Insert into {} values (".format(tablename))
	
	#處理到倒數第二行
  for i in data[:-2]:
    result.append(genSQLValues(i, colIndex))
	
	#最後一行要用 ); 所以獨立處理
  lastValue = genSQLValues(data[-1], colIndex)
  result.append(lastValue[:-1] + ");\n")
  return result
 

主程式

In [ ]:
5ROOT = '/content/drive' #根目錄位址
drive.mount(ROOT) #登入google drive
PROJ = "My Drive/DataScience/110-2/Final/410510008" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)
!mkdir -p "{PROJECT_PATH}"  #這行的意義??

TAG_RE = re.compile(r'<[^>]+>') #正則表達式??
code=["146626"]
studentId="410510008"
  
for urlCode in code:
  URL = "https://data.gov.tw/dataset/{}".format(urlCode)
  result = requests.get(URL)
  soup = bs(result.text, "lxml")

  target = soup.select('script[type="application/ld+json"]')
  targetJson = json.loads(TAG_RE.sub('', str(target[0])))

  #印出下載點
  name = targetJson[1]["name"]
  print(urlCode+"-"+name)
  filepath = targetJson[1]["distribution"][0]["contentUrl"]
  print(filepath)
    
  #像下載點發出request 
  data = getCSV_From_URL(filepath)
    
  #將檔案存入雲端中
  myFile = join(PROJECT_PATH, urlCode+"-"+name+".html") # join(路徑, 檔案名稱)
  with open(myFile, 'wb') as file:  #open(儲存路徑, 'w'=write , encoding編碼)
    file.write(data.content)  # text跟content的差別???
    file.close
    
  dataList = openFile(myFile)
  limit = int(input("請輸入要顯示前面多少行："))
  print_header_and_few_line(dataList, limit)

  colIndexList = getColumnInedex()
  colIndexList.sort
  print("\n你選擇的欄位為: {}".format(colIndexList))
  dataList = dataList[1:] #去除header

  tabName = input("你要輸入的資料庫檔名是：")
  sqlList = gen_SQL_insert_cmd(tabName,dataList,colIndexList)

  #印出SQL語法的前三行與末三行
  print("產生資料的前三行與末三行為：")
  print(sqlList[:3])
  print()
  print(sqlList[-4:-1])
    
  # 將資料寫入一個SQL檔案
  print("資料寫入檔案：{}.sql".format(studentId))
  mySQLFile = join(PROJECT_PATH, studentId+".sql")
  with open(mySQLFile, 'w') as f:
    for item in sqlList:
      f.write("{}\n".format(item))

Mounted at /content/drive
146626-嬰兒出生數按嬰兒性別及生父母年齡分（按發生）
https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=596EA333-62C1-4035-B88D-8E38C4447A54
請輸入要顯示前面多少行：3
['\ufeffstatistic_yyy', 'according', 'district_code', 'site_id', 'sex', 'father_age', 'mother_age', 'birth_count']
['統計年度', '按照別', '區域別代碼', '區域別', '嬰兒性別', '生父年齡', '生母年齡', '嬰兒出生數']
['109', '按發生日期分', '65000010', '新北市板橋區', '男', '未滿20歲', '未滿15歲', '0']
['109', '按發生日期分', '65000010', '新北市板橋區', '男', '未滿20歲', '15～19歲', '0']
請依序輸入你想要的欄位索引值, 輸入 -1 結束：1
請依序輸入你想要的欄位索引值, 輸入 -1 結束：2
請依序輸入你想要的欄位索引值, 輸入 -1 結束：3
請依序輸入你想要的欄位索引值, 輸入 -1 結束：4
請依序輸入你想要的欄位索引值, 輸入 -1 結束：5
請依序輸入你想要的欄位索引值, 輸入 -1 結束：6
請依序輸入你想要的欄位索引值, 輸入 -1 結束：7
請依序輸入你想要的欄位索引值, 輸入 -1 結束：-1

你選擇的欄位為: [1, 2, 3, 4, 5, 6, 7]


open 讀寫模式: https://www.796t.com/content/1550169482.html